# Cleaner after all projects combined

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Show all columns
pd.set_option('display.max_columns', None)


Read JSON file

In [ ]:
df = pd.read_json('../data/kickstarter.json', lines = True)
df.shape

Check data head

In [ ]:
df.head()

# Clean column types

## Change all dates to datetime

In [ ]:
date_columns = ['profile_state_changed_at', 'created_at', 'launched_at', 'deadline', 'state_changed_at']
df[date_columns] = df[date_columns].apply(pd.to_datetime, unit='s')

## Reorder Data

In [ ]:
df = df.sort_values(by=['creator_id','launched_at'], ascending=True).reset_index(drop = True)
df[['creator_id','id','state','launched_at', 'deadline', 'pledged', 'goal']].head(20)

# Create Variables

## Create dataframe for 1st project only

In [ ]:
# Copy dataframe
base = df.copy()

# First, let's flag the first project for each creator
base['is_first_project'] = base.groupby('creator_id').cumcount() == 0

# Now, we filter out rows where 'is_first_project' is False
base = base[base['is_first_project'] == True].sort_values(by=['creator_id','launched_at'], ascending=True).reset_index(drop = True)

# Set creator id as index
base.set_index('creator_id', inplace=True)

## Create dataframe for 2nd project only

In [ ]:
# copy dataframe
next_campaign = df.copy()

# Get all the entrepreneurs' next campaign
next_campaign['is_second_project'] = next_campaign.groupby('creator_id').cumcount() == 1

# Filter out where is_second_project is True
next_campaign = next_campaign[next_campaign['is_second_project'] == True].sort_values(by=['creator_id','launched_at'], ascending=True).reset_index(drop = True)

# Check head
next_campaign[['creator_id','id','state','launched_at', 'deadline', 'pledged', 'goal']].head()

## Variables for Next Campaign

In [ ]:
# Create dummy for success, failed, and canceled
next_campaign['success_next'] = np.where(next_campaign['state'] == 'successful', 1, 0)
next_campaign['failed_next'] = np.where(next_campaign['state'] == 'failed', 1, 0)
next_campaign['canceled_next'] = np.where(next_campaign['state'] == 'canceled', 1, 0)

# Create column raised next
next_campaign['raised_next'] = np.where(next_campaign['success_next'] == 1, next_campaign['pledged'], 0)

# Map variables to first campaign
next_campaign = next_campaign[['creator_id', 'success_next', 'failed_next', 'canceled_next', 'raised_next']]
# Set creator id as index
next_campaign.set_index('creator_id', inplace=True)


In [ ]:
# Merge with base
base = base.merge(next_campaign, on = 'creator_id', how='left')

In [ ]:
base[['id', 'state', 'goal', 'pledged', 'raised_next', 'success_next', 'failed_next', 'canceled_next']].head()

## Variables for all campaigns after first

In [ ]:
# Copy dataframe
test = df.copy()

# First, let's flag the first project for each creator
test['is_first_project'] = test.groupby('creator_id').cumcount() == 0

# Now, we filter out rows where 'is_first_project' is False
test = test[test['is_first_project'] == False]

# Append the first project launched at colmn by each creator as new column 
test = test.merge(df.groupby('creator_id')['launched_at'].first().reset_index().rename(columns={'launched_at':'first_project_launched_at'}), on='creator_id')

# Reset index if needed
test = test.reset_index(drop=True)

test[['creator_id','id','state', 'state_changed_at','launched_at', 'deadline', 'pledged', 'goal', 'is_first_project', 'first_project_launched_at']].head()

Create key variables

In [ ]:
# Calculate total number of Kickstarter campaigns for each creator after first one
test.groupby('creator_id', as_index = False)['id'].count().sort_values(by = 'creator_id', ascending = True).rename(columns = {'id':'total_campaigns_after_first'}).head()

In [ ]:
# Calculate total number of successful, failed, canceled Kickstarter campaigns for each creator after first one
successful = dict(test[test['state'] == 'successful'].groupby('creator_id')['id'].count().rename('total_successful_campaigns_after_first'))
failed = dict(test[test['state'] == 'failed'].groupby('creator_id')['id'].count().rename('total_failed_campaigns_after_first'))
canceled = dict(test[test['state'] == 'canceled'].groupby('creator_id')['id'].count().rename('total_canceled_campaigns_after_first'))
total = dict(test.groupby('creator_id')['id'].count().rename('total_campaigns_after_first'))
total_raised = dict(test[test['state'] == 'successful'].groupby('creator_id')['pledged'].sum().rename('total_raised_after_first'))


# Calculate total number of successful, failed, canceled Kickstarter campaigns for each creator after first one for the next 3 years
successful_3 = dict(test[(test['state'] == 'successful') & (test['launched_at'] <= test['first_project_launched_at'] + pd.DateOffset(years=3))].groupby('creator_id')['id'].count().rename('total_successful_campaigns_after_first_3'))
failed_3 = dict(test[(test['state'] == 'failed') & (test['launched_at'] <= test['first_project_launched_at'] + pd.DateOffset(years=3))].groupby('creator_id')['id'].count().rename('total_failed_campaigns_after_first_3'))
canceled_3 = dict(test[(test['state'] == 'canceled') & (test['launched_at'] <= test['first_project_launched_at'] + pd.DateOffset(years=3))].groupby('creator_id')['id'].count().rename('total_canceled_campaigns_after_first_3'))
total_3 = dict(test[test['launched_at'] <= test['first_project_launched_at'] + pd.DateOffset(years=3)].groupby('creator_id')['id'].count().rename('total_campaigns_after_first_3'))
total_raised_3 = dict(test[(test['state'] == 'successful') & (test['launched_at'] <= test['first_project_launched_at'] + pd.DateOffset(years=3))].groupby('creator_id')['pledged'].sum().rename('total_raised_after_first_3'))


Create dataset of entrepreneurs' first projects only

In [ ]:
# Map outcomes successful, failed, canceled to new dataframe
base['total_successful_after_first'] = base.index.map(successful)
base['total_failed_after_first'] = base.index.map(failed)
base['total_canceled_after_first'] = base.index.map(canceled)
base['total_campaigns_after_first'] = base.index.map(total)
base['total_raised_after_first'] = base.index.map(total_raised)
# Map outcomes successful, failed, canceled to new dataframe for the next 3 years
base['total_successful_after_first_3'] = base.index.map(successful_3)
base['total_failed_after_first_3'] = base.index.map(failed_3)
base['total_canceled_after_first_3'] = base.index.map(canceled_3)
base['total_campaigns_after_first_3'] = base.index.map(total_3)
base['total_raised_after_first_3'] = base.index.map(total_raised_3)


base[['is_first_project', 'total_successful_after_first', 'total_failed_after_first', 'total_canceled_after_first', 'total_campaigns_after_first', 'total_raised_after_first', 'total_successful_after_first_3', 'total_failed_after_first_3', 
         'total_canceled_after_first_3', 'total_campaigns_after_first_3', 'total_raised_after_first_3']].head()

In [ ]:
# Dump created dataframe to json
base.to_json('..\data\creator_first_project.json', orient='records', lines=True)